In [16]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier

In [17]:
# Read data from file
hhold = pd.read_csv('train/A_hhold_train.csv')
indiv = pd.read_csv('train/A_indiv_train.csv')

In [18]:
hhold.head()

,id,wBXbHZmp,SlDKnCuu,KAJOWiiw,DsKacCdL,rtPrBBPl,tMJrvvut,jdetlNNF,maLAYXwi,vwpsXRGk,...,sDGibZrP,CsGvKKBJ,OLpGAaEu,LrDrWRjC,JCDeZBXq,HGPWuGlV,GDUPaBQs,WuwrCsIY,AlDbXTlZ,country
0,46107,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,XAmOF,...,JqHnW,MaXfS,etZsD,idRwx,LPtkN,vkbkA,qQxrL,AITFl,aQeIm,A
1,82739,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,lwCkE,...,JqHnW,MaXfS,HxnJy,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq,A
2,9646,JhtDR,GUusz,BIZns,ZYabk,uxuSS,PHMVg,NDTCU,sehIp,qNABl,...,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq,A
3,10975,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,sPNOc,...,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq,A
4,16463,JhtDR,alLXR,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,NdlDR,...,JqHnW,MaXfS,etZsD,idRwx,UyAms,vkbkA,qQxrL,GAZGl,aQeIm,A


In [19]:
indiv.head()

,id,iid,HeUgMnzF,CaukPfUC,MzEtIdUF,gtnNTNam,SWoXNmPc,eXbOkwhI,OdXpbPGJ,XONDGWjH,...,ukWqmeSS,qqVibbSA,MgCoFhXK,rFpoTXAq,RXcLsVAQ,rQWIpTiG,XizJGmbu,xqUooaNJ,poor,country
0,80389,1,XJsPz,mOlYV,UFoKR,SSvEP,onRNG,YXCNt,4.0,oArAw,...,181,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,dSJoN,True,A
1,80389,2,XJsPz,mOlYV,axSTs,CXizI,onRNG,YXCNt,4.0,ccbZA,...,141,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,JTCKs,True,A
2,80389,3,TRFeI,mOlYV,axSTs,CXizI,NDnCs,YXCNt,4.0,fOUHD,...,41,QQdHS,gCSRj,Hikoa,zQvdC,rkLqZ,juMSt,JTCKs,True,A
3,80389,4,XJsPz,yAyAe,FRcdT,CXizI,onRNG,YXCNt,NaN,fOUHD,...,16,QQdHS,uEstx,Hikoa,zQvdC,jVHyH,GtHel,JTCKs,True,A
4,39883,1,XJsPz,mOlYV,UFoKR,HIvIU,onRNG,YXCNt,4.0,oArAw,...,381,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,UaIsy,False,A


In [20]:
def col_mapper (df):
    '''Creates dictionary of original column names (key) and their
    altered names (value)'''
    # Initialize counters and temporary dictionary
    num_counter = 1
    cat_counter = 1
    temp = {}
    # Iterate through columns
    for col in df.columns:
        # For protected columns, add to dictionary as is
        if col in ['id', 'iid', 'poor', 'country']:
            temp[col] = col
        else:
            # For numeric, rename like "num_005"
            if df[col].dtype in ['int64', 'float64']:
                temp[col] = 'num' + '_' + str(num_counter).zfill(3)
                num_counter += 1
            # For categorical, rename like "cat_006"
            elif df[col].dtype == 'object':
                temp[col] = 'cat' + '_' + str(cat_counter).zfill(3)
                cat_counter += 1
    return temp

In [21]:
# Create a dictionary of original --> new column names
hhold_column_dict = col_mapper(hhold)
# Also create a reversed dictionary for easy look-up in other direction
hhold_rev_column_dict = {val: key for key, val in hhold_column_dict.items()}

# Same for indiv
indiv_column_dict = col_mapper(indiv)
# Also create a reversed dictionary for easy look-up in other direction
indiv_rev_column_dict = {val: key for key, val in indiv_column_dict.items()}

In [22]:
# Make a set of new columns
hhold_new_columns = hhold.columns.map(lambda x: hhold_column_dict[x])
indiv_new_columns = indiv.columns.map(lambda x: indiv_column_dict[x])

In [23]:
# Replace old columns with new
hhold.columns = hhold_new_columns
indiv.columns = indiv_new_columns

In [24]:
# Drop country as variable
hhold.drop(labels=['country'], axis=1, inplace=True)
indiv.drop(labels=['country'], axis=1, inplace=True)

In [25]:
hhold.head()

,id,cat_001,cat_002,cat_003,cat_004,cat_005,cat_006,cat_007,cat_008,cat_009,...,cat_330,cat_331,cat_332,cat_333,cat_334,cat_335,cat_336,cat_337,cat_338,cat_339
0,46107,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,XAmOF,...,wxJis,JqHnW,MaXfS,etZsD,idRwx,LPtkN,vkbkA,qQxrL,AITFl,aQeIm
1,82739,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,lwCkE,...,AZwXA,JqHnW,MaXfS,HxnJy,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq
2,9646,JhtDR,GUusz,BIZns,ZYabk,uxuSS,PHMVg,NDTCU,sehIp,qNABl,...,AZwXA,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq
3,10975,JhtDR,GUusz,TuovO,ZYabk,feupP,PHMVg,NDTCU,sehIp,sPNOc,...,wxJis,JqHnW,MaXfS,USRak,idRwx,UyAms,vkbkA,qQxrL,AITFl,cecIq
4,16463,JhtDR,alLXR,TuovO,ZYabk,feupP,PHMVg,NDTCU,cLAGr,NdlDR,...,wxJis,JqHnW,MaXfS,etZsD,idRwx,UyAms,vkbkA,qQxrL,GAZGl,aQeIm


In [26]:
indiv.head()

,id,iid,cat_001,cat_002,cat_003,cat_004,cat_005,cat_006,num_001,cat_007,...,cat_031,num_002,cat_032,cat_033,cat_034,cat_035,cat_036,cat_037,cat_038,poor
0,80389,1,XJsPz,mOlYV,UFoKR,SSvEP,onRNG,YXCNt,4.0,oArAw,...,yOwsR,181,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,dSJoN,True
1,80389,2,XJsPz,mOlYV,axSTs,CXizI,onRNG,YXCNt,4.0,ccbZA,...,yOwsR,141,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,JTCKs,True
2,80389,3,TRFeI,mOlYV,axSTs,CXizI,NDnCs,YXCNt,4.0,fOUHD,...,yOwsR,41,QQdHS,gCSRj,Hikoa,zQvdC,rkLqZ,juMSt,JTCKs,True
3,80389,4,XJsPz,yAyAe,FRcdT,CXizI,onRNG,YXCNt,NaN,fOUHD,...,yOwsR,16,QQdHS,uEstx,Hikoa,zQvdC,jVHyH,GtHel,JTCKs,True
4,39883,1,XJsPz,mOlYV,UFoKR,HIvIU,onRNG,YXCNt,4.0,oArAw,...,yOwsR,381,QQdHS,uEstx,Hikoa,zQvdC,xUYIC,juMSt,UaIsy,False


In [58]:
# Get various lists of features for easy pandas slicing
# Indices and target variable are in protected class
protected_cols = ['id', 'iid', 'poor']
# Everything not protected
all_feat_list = [col for col in indiv.columns if col not in protected_cols]
# Numeric
num_feat_list = [col for col in indiv.columns if indiv[col].dtype in ['int64', 'float64'] and col not in protected_cols]
# Categorical
cat_feat_list = [col for col in indiv.columns if indiv[col].dtype == 'object' and col not in protected_cols]

In [59]:
# Return a new dataframe with all categorical features converted into dummies
indiv_dummies = pd.get_dummies(indiv[cat_feat_list])

In [61]:
indiv_dummies.head()

,cat_001_BNCcM,cat_001_HUpWg,cat_001_JMXQx,cat_001_PAVsH,cat_001_SJPkb,cat_001_SlRmt,cat_001_TRFeI,cat_001_XJgvq,cat_001_XJsPz,cat_001_YsSBt,...,cat_036_rkLqZ,cat_036_xUYIC,cat_037_FUUXv,cat_037_GtHel,cat_037_juMSt,cat_038_ALcKg,cat_038_JTCKs,cat_038_UaIsy,cat_038_dSJoN,cat_038_vhhVz
0,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,0,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,1,0,0,0
2,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,1,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,0,1,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,1,0,0,1,0,0,1,0,0


In [62]:
# Merge the protected and numeric columns with the dummy categorical data
# We keep this named separately from indiv to avoid overwriting original data
indiv_dummies = pd.merge(indiv[protected_cols + num_feat_list], indiv_dummies, left_index=True, right_index=True)

In [63]:
indiv_dummies.head()

,id,iid,poor,num_001,num_002,cat_001_BNCcM,cat_001_HUpWg,cat_001_JMXQx,cat_001_PAVsH,cat_001_SJPkb,...,cat_036_rkLqZ,cat_036_xUYIC,cat_037_FUUXv,cat_037_GtHel,cat_037_juMSt,cat_038_ALcKg,cat_038_JTCKs,cat_038_UaIsy,cat_038_dSJoN,cat_038_vhhVz
0,80389,1,True,4.0,181,0,0,0,0,0,...,0,1,0,0,1,0,0,0,1,0
1,80389,2,True,4.0,141,0,0,0,0,0,...,0,1,0,0,1,0,1,0,0,0
2,80389,3,True,4.0,41,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,0
3,80389,4,True,NaN,16,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
4,39883,1,False,4.0,381,0,0,0,0,0,...,0,1,0,0,1,0,0,1,0,0


In [71]:
# Test for missing values in numeric features - here we see about 6,000
indiv_dummies['num_001'].isnull().sum()

6268

In [72]:
# The second numeric feature is clean, though
indiv_dummies['num_002'].isnull().sum()

0

In [73]:
# There is an extremely high bias for 4.0 in 'num_001', so we will use mode 
# rather than the mean to replace NaNs
indiv_dummies['num_001'].value_counts()

4.0      29436
100.0      314
84.0       273
88.0        92
64.0        74
116.0       69
20.0        62
124.0       57
172.0       54
24.0        51
74.0        49
104.0       45
76.0        41
28.0        39
94.0        36
148.0       35
112.0       35
16.0        32
44.0        32
52.0        31
144.0       31
18.0        27
102.0       25
54.0        20
40.0        19
14.0        19
6.0         16
60.0        16
22.0        15
96.0        14
         ...  
90.0         4
108.0        4
46.0         4
120.0        4
114.0        3
128.0        3
110.0        3
184.0        3
86.0         2
56.0         2
140.0        2
134.0        2
156.0        2
186.0        2
38.0         2
122.0        2
214.0        1
106.0        1
42.0         1
5.0          1
48.0         1
180.0        1
152.0        1
200.0        1
66.0         1
196.0        1
30.0         1
50.0         1
118.0        1
178.0        1
Name: num_001, Length: 82, dtype: int64

In [79]:
# Fill missing values with the mode of 'num_001'
indiv_dummies['num_001'] = indiv_dummies['num_001'].fillna(value=indiv_dummies['num_001'].mode().item())

In [95]:
# Get a list of dummy features
dummy_feat_list = [col for col in indiv_dummies.columns if col not in protected_cols]
# Separate data into independent and dependent variables
X = indiv_dummies[dummy_feat_list]
Y = indiv_dummies['poor']

In [92]:
# Fit forest
forest = RandomForestClassifier(n_estimators=10000, n_jobs=-1)
forest.fit(X, Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10000, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [93]:
# Gather feature importances
importances = forest.feature_importances_

In [96]:
# Gather ranked indices of feature importances
indices = np.argsort(importances)[::-1]
for f in range(len(dummy_feat_list)):
    print('{0:3d} ({1:.4f}) {2}'.format(f + 1, importances[indices[f]], dummy_feat_list[indices[f]]))

  1 (0.2210) num_002
  2 (0.0280) cat_014_BQEnF
  3 (0.0241) cat_014_flBEG
  4 (0.0186) cat_014_VGNER
  5 (0.0169) cat_023_Qydia
  6 (0.0169) cat_023_vtkRP
  7 (0.0164) cat_010_scxJu
  8 (0.0146) cat_002_kzSFB
  9 (0.0128) cat_014_iyqBV
 10 (0.0114) cat_017_OeQKE
 11 (0.0110) cat_022_AyuSE
 12 (0.0108) cat_010_bJTYb
 13 (0.0104) cat_033_gCSRj
 14 (0.0104) cat_002_mOlYV
 15 (0.0102) cat_022_hCKQi
 16 (0.0101) num_001
 17 (0.0099) cat_014_lcEtN
 18 (0.0095) cat_019_dpMMl
 19 (0.0092) cat_018_CneHb
 20 (0.0092) cat_033_uEstx
 21 (0.0092) cat_014_bszTA
 22 (0.0087) cat_036_rkLqZ
 23 (0.0086) cat_017_XNPgB
 24 (0.0086) cat_036_xUYIC
 25 (0.0085) cat_012_HHynv
 26 (0.0084) cat_012_yhUHu
 27 (0.0083) cat_007_fOUHD
 28 (0.0082) cat_012_DgtXD
 29 (0.0080) cat_010_tqINY
 30 (0.0080) cat_012_GmSKW
 31 (0.0078) cat_013_pdgUV
 32 (0.0075) cat_004_HIvIU
 33 (0.0074) cat_012_YEngm
 34 (0.0072) cat_012_EaHvf
 35 (0.0071) cat_027_WNISg
 36 (0.0071) cat_012_AYcgs
 37 (0.0070) cat_019_SjaWF
 38 (0.0069) 

In [103]:
# Gather a list of the top features
top_features = dummy_feat_list[np.where(importances > 0.0092)]
top_features

array(['num_001', 'num_002', 'cat_002_kzSFB', 'cat_002_mOlYV',
       'cat_010_bJTYb', 'cat_010_scxJu', 'cat_014_BQEnF', 'cat_014_VGNER',
       'cat_014_bszTA', 'cat_014_flBEG', 'cat_014_iyqBV', 'cat_014_lcEtN',
       'cat_017_OeQKE', 'cat_018_CneHb', 'cat_019_dpMMl', 'cat_022_AyuSE',
       'cat_022_hCKQi', 'cat_023_Qydia', 'cat_023_vtkRP', 'cat_033_gCSRj',
       'cat_033_uEstx'],
      dtype='<U13')